# ファンクターからアプリカティブファンクターへ
## ファンクターとは
関数で写せるもののこと。リスト,Maybe,木,I/Oアクション(結果を関数で写せば良い)など。メソッドはfmapのみ。Functorのインスタンスになるには、型コンストラクタの種類(kind)は`*->*`でないといけない、つまり、型変数として具体型をただ1つ持つ必要がある。

In [11]:
import Data.Char
import　Data.List
main = do 
    line <- fmap (intersperse '-' . reverse . map toUpper) getLine
    putStrLn line

## Functorとしての関数
関数の型を表す`->`は中置型になっているが、2つの型引数を取る型コンストラクタでもある。fmapは関数合成を意味する。
```
fmap :: (a -> b) -> (r -> a) -> (r -> b)
```

In [15]:
:m + Control.Monad.Instances
:t fmap (*3) (+100)
:t fmap (*2)
:t fmap (replicate 3)

fmap (*3) (+100) :: forall b. Num b => b -> b

fmap (*2) :: forall (f :: * -> *) b. (Functor f, Num b) => f b -> f b

fmap (replicate 3) :: forall (f :: * -> *) a. Functor f => f a -> f [a]

## Fmapの二つの考え方
- 関数とファンクター値をとって、その関数でファンクター値を移して返すもの
- 値から値への関数をとって、それをファンクター値からファンクター値への関数に持ち上げたものを返す関数

In [20]:
fmap (replicate 3) [1,2,3,4]
fmap (replicate 3) (Just 4)
fmap (replicate 3) (Right "foo")
fmap (replicate 3) Nothing
fmap (replicate 3) (Left "bar")

[[1,1,1],[2,2,2],[3,3,3],[4,4,4]]

Just [4,4,4]

Right ["foo","foo","foo"]

Nothing

Left "bar"

## Fanctor則
- idでファンクター値を移した場合、ファンクター値が変化してはいけない(fmap id = id)
- 関数合成の法則:fmap (f.g) = fmap f. fmap g
この二つを満たしているとよい。

## Applicative(ファンクター)
ファンクターでは通常の関数でファンクターの中の値を写すことができるが、ファンクターの中の関数で別のファンクターの中の値を写すことをしたい。

関数は以下の2つ。（型コンストラクタをApplicativeに属するにはまずFuncterに属させる必要があるので、fmapもできる。）
- pure：値をとって、内部にその値を含むapplicative値に包む。
- `<*>`:関数の入っているファンクター値と値の入っているファンクター値を引数にとって、適用。左結合の関数。

In [27]:
Just (+3) <*> Just 9
pure (+3) <*> Just 10
(+3) <$> Just 10 -- f <$> x= fmap f x = pure f <*> x
Nothing <*> Just 11
Just (+) <*> Just 3 <*> Just 5

Line 2: Use <$>
Found:
pure (+ 3) <*> Just 10
Why not:
(+ 3) <$> Just 10

Just 12

Just 13

Just 13

Nothing

Just 8

In [28]:
[(*0),(+100),(^2)] <*> [1,2,3]

[0,0,0,101,102,103,1,4,9]

In [29]:
(*) <$> [2,5,10] <*> [8,10,11]

[16,20,22,40,50,55,80,100,110]

applicativeなものにはMaybe,List,IO,関数(->)などがある。

I/O

In [33]:
myAction :: IO String
myAction = (++) <$> getLine <*> getLine

関数`->`
```
instance Applicative ((->) r) where
    pure x = (_ -> x)
    f <*> g = \x -> f x (g x)
```
以下のコードではまず引数を`+3`と`*100`にわたし、その結果をたす。

In [34]:
(+) <$> (+3) <*> (*100) $ 5
(\x y z -> [x,y,z]) <$> (+3) <*> (*2) <*> (/2) $ 5

508

[8.0,10.0,2.5]

In [37]:
import Control.Applicative
getZipList $ (+) <$> ZipList [1,2,3] <*> ZipList [100,100,100]
getZipList $ (+) <$> pure 100 <*> ZipList [100,100,100]

Line 3: Redundant <$>
Found:
(+) <$> pure 100
Why not:
pure ((+) 100)

[101,102,103]

[200,200,200]

## アプリカティブ則
- `pure f <*> x = fmap f x`
- `pure id <*> v = v`
- `pure (.) <*> u <*> v <*> w =u<*> (v<*> w)`
- `pure f <*> pure x = pure (f x)`
- `u <*> pure y = pure ($ y) <*> u`